In [1]:
import pandas as pd

In [2]:
raw = pd.read_csv("../00_data/01_raw/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv")

In [5]:
raw

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,2.978361e+05
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,4.849451e+05
2,3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,1.067059e+05
3,4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,4.818284e+05
4,5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,6.876848e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220735,220736,7.803100e+11,7.803100e+11,78,30,961000,2,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,3414.446949,3.355857e+05
220736,220737,7.803100e+11,7.803100e+11,78,30,961000,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,2421.025608,2.924305e+05
220737,220738,7.803100e+11,7.803100e+11,78,30,961000,5,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,1955.909418,1.619395e+05
220738,220739,7.803100e+11,7.803100e+11,78,30,960700,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,10,1,4.000000,16896.768870,1.038966e+07


In [15]:
D3_RENAME = {
    "D3A":    "road_network",
    "D3AAO":  "auto_net",
    "D3AMM":  "multimodal_net",
    "D3APO":  "pedestrian_net",
    "D3B":    "street_intersection", # (weighted, auto-oriented removed)
    "D3BAO":  "auto_intersections",
    "D3BMM3": "multimodal_3leg_intersections",
    "D3BMM4": "multimodal_4leg_intersections",
    "D3BPO3": "pedestrian_3leg_intersections",
    "D3BPO4": "pedestrian_4leg_intersections",
    "Ac_Land": "Ac_Land"
}

df_select = raw[list(D3_RENAME.keys())+['CBSA']].rename(columns=D3_RENAME)

In [16]:
# Build totals: (density * area)
density_cols = [c for c in D3_RENAME.values() if c != "Ac_Land"]
for c in density_cols:
    df_select[c + "_total"] = df_select[c] * df_select["Ac_Land"]

# Aggregate to CBSA
df_cbsa = (
    df_select
      .groupby('CBSA', as_index=False)
      .agg({**{c + "_total": "sum" for c in density_cols}, "Ac_Land": "sum"})
)


In [17]:
df_cbsa

,CBSA,road_network_total,auto_net_total,multimodal_net_total,pedestrian_net_total,street_intersection_total,auto_intersections_total,multimodal_3leg_intersections_total,multimodal_4leg_intersections_total,pedestrian_3leg_intersections_total,pedestrian_4leg_intersections_total,Ac_Land
0,10100.0,3.284660e+06,405321.770783,2.646972e+06,2.323662e+05,2.253482e+06,4.416000e+05,8.025600e+05,824960.000122,6.022400e+05,4.915200e+05,1.816927e+06
1,10140.0,2.374547e+06,104440.891567,1.599555e+05,2.110150e+06,2.589727e+06,1.440000e+05,4.236800e+05,113919.999991,2.356480e+06,6.214400e+05,1.216910e+06
2,10180.0,4.222981e+06,649977.555417,1.960052e+06,1.612951e+06,6.348198e+06,1.219200e+06,2.706560e+06,566400.000211,3.808640e+06,1.436160e+06,1.755788e+06
3,10220.0,1.003198e+06,118841.502159,5.561159e+05,3.282406e+05,1.655392e+06,2.086400e+05,8.627200e+05,156159.999958,8.332800e+05,3.680000e+05,4.610480e+05
4,10300.0,1.318877e+06,241518.103246,8.121703e+05,2.651885e+05,1.677814e+06,4.224000e+05,8.582400e+05,213120.000005,9.030400e+05,2.899200e+05,4.797457e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
933,49660.0,4.360935e+06,806578.554084,1.665215e+06,1.889141e+06,9.193127e+06,8.960000e+05,3.553280e+06,935679.999969,5.907840e+06,1.946880e+06,1.089320e+06
934,49700.0,2.180572e+06,336653.884516,1.065351e+05,1.737383e+06,3.295718e+06,5.132800e+05,3.641600e+05,113280.000069,3.271040e+06,7.577600e+05,7.900925e+05
935,49740.0,2.408522e+06,217941.354627,1.111215e+06,1.079365e+06,4.359259e+06,1.484800e+05,1.288960e+06,476159.999328,3.207680e+06,8.838400e+05,3.528780e+06
936,49780.0,1.206591e+06,188083.838884,7.821741e+05,2.363330e+05,1.744664e+06,2.924800e+05,1.165440e+06,162560.000015,8.256000e+05,2.540800e+05,4.253001e+05


In [ ]:
df_cbsa.to_csv('../00_data/02_intermediate/road_cbsa.csv',index=False)